In [1]:
import os
import urllib.request

In [2]:
from gromacs_py import gmx
gmx.show_log()
from gromacs_py.tools import ambertools
ambertools.show_log()

from pdb_manip_py import pdb_manip

In [3]:
ls

00_basic_example.ipynb                 05_test_ligand_ambertools_trypsin.ipynb
01_monitor_test.ipynb                  data_lig_dna/
02_cyclic_amber.ipynb                  data_sim/
03_test_ligand_ambertools-Copy1.ipynb  phenol_h.mol2
03_test_ligand_ambertools.ipynb        phenol_h.pdb
04_test_ligand_dna_ambertools.ipynb    phenol.pdb


In [4]:
cd /home/murail/Documents/Code/gromacs_py/docs/notebook/

/home/murail/Documents/Code/gromacs_py/docs/notebook


In [5]:
rm -r data_lig_trypsin

rm: cannot remove 'data_lig_trypsin': No such file or directory


- 453D -> 1:20 m
- 1D30 -> 27 s

In [6]:
DATA_OUT = 'data_lig_trypsin'
PDB_ID = '1GHZ'
os.makedirs(DATA_OUT, exist_ok = True)

raw_pdb = urllib.request.urlretrieve('http://files.rcsb.org/download/{}.pdb'.format(PDB_ID),
                           '{}/{}.pdb'.format(DATA_OUT, PDB_ID))

In [7]:
# Extract first conf:

#pdb_multi = pdb_manip.Multi_Coor(raw_pdb[0])
#pdb_model_1 = '{}/{}_model_1.pdb'.format(DATA_OUT, PDB_ID)
#pdb_multi.coor_list[0].write_pdb(pdb_model_1)

pdb_model_1 = '{}/{}.pdb'.format(DATA_OUT, PDB_ID)

In [8]:
md_sys = gmx.GmxSys(name=PDB_ID, coor_file=pdb_model_1)
md_sys.prepare_top(out_folder=os.path.join(DATA_OUT, 'prot_top'),
                   ff='amber99sb-ildn', include_mol={'120': 1})

Succeed to read file ../1GHZ.pdb ,  4193 atoms found
Succeed to read file ../1GHZ.pdb ,  4193 atoms found
Succeed to save file tmp_pdb2pqr.pdb
pdb2pqr_cli --ff AMBER --ffout AMBER --chain --ph-calc-method=propka --with-ph=7.00 tmp_pdb2pqr.pdb 00_1GHZ.pqr
Succeed to read file 00_1GHZ.pqr ,  3221 atoms found
residue(s) CA not included, add this residue in ...
residue(s) MG not included, add this residue in ...
Succeed to read file ../1GHZ.pdb ,  4193 atoms found
Succeed to save file 120.pdb
reduce -build -nuclear 120.pdb
Succeed to save file 120_h.pdb
Succeed to read file 120_h.pdb ,  31 atoms found
Succeed to save file 120_h_unique.pdb
acpype -i 120_h_unique.pdb -b 120 -c bcc -a gaff -o gmx -n 1
Succeed to create topologie in 120
120
residue(s) CYS not included, add this residue in ...
residue(s) SO4 not included, add this residue in ...
Chain: A  Residue: 0 to 219
Succeed to save file 01_1GHZ_good_his.pdb
-Create topologie
gmx pdb2gmx -f 01_1GHZ_good_his.pdb -o 1GHZ_pdb2gmx.pdb -p 1GHZ

In [9]:
md_sys.em_2_steps(out_folder=os.path.join(DATA_OUT, 'prot_em'),
                  no_constr_nsteps=10000,
                  constr_nsteps=10000,
                  posres="",
                  emtol=0.1, nstxout=100, maxwarn=3,
                  create_box_flag=True)

-Create pbc box
gmx editconf -f data_lig_trypsin/prot_top/1GHZ_pdb2gmx_mol.pdb -o data_lig_trypsin/prot_top/1GHZ_pdb2gmx_mol_box.pdb -bt dodecahedron -d 1.0
WARNING !!! ADDING unusual parameter : "emtol"in the mdp file None
-Create the tpr file Init_em_1GHZ.tpr
gmx grompp -f Init_em_1GHZ.mdp -c ../prot_top/1GHZ_pdb2gmx_mol_box.pdb -r ../prot_top/1GHZ_pdb2gmx_mol_box.pdb -p ../prot_top/1GHZ_pdb2gmx_mol.top -po out_Init_em_1GHZ.mdp -o Init_em_1GHZ.tpr -maxwarn 3
-Launch the simulation Init_em_1GHZ.tpr
gmx mdrun -s Init_em_1GHZ.tpr -deffnm Init_em_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



WARNING !!! ADDING unusual parameter : "emtol"in the mdp file Init_em_1GHZ.mdp
-Create the tpr file 1GHZ.tpr
gmx grompp -f 1GHZ.mdp -c Init_em_1GHZ.gro -r Init_em_1GHZ.gro -p ../prot_top/1GHZ_pdb2gmx_mol.top -po out_1GHZ.mdp -o 1GHZ.tpr -maxwarn 3
-Launch the simulation 1GHZ.tpr
gmx mdrun -s 1GHZ.tpr -deffnm 1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [10]:
view = md_sys.view_coor()
view

_ColormakerRegistry()

-Convert trj/coor
gmx trjconv -f data_lig_trypsin/prot_em/1GHZ.gro -o data_lig_trypsin/prot_em/1GHZ_compact.pdb -s data_lig_trypsin/prot_em/1GHZ.tpr -ur compact -pbc none
Succeed to read file data_lig_trypsin/prot_em/1GHZ_compact.pdb ,  3252 atoms found


NGLWidget()

In [11]:
md_sys.solvate_add_ions(out_folder=os.path.join(DATA_OUT, 'sys_top'),
                        ion_C=0.15, maxwarn=3, create_box_flag=False)
md_sys.display()


-Solvate the pbc box
Copy topologie file and dependancies
Copy topologie file and dependancies
-Create the tpr file genion_1GHZ_water_ion.tpr
gmx grompp -f ../../../../gromacs_py/template/mini.mdp -c 1GHZ_water.pdb -r 1GHZ_water.pdb -p 1GHZ_water_ion.top -po out_mini.mdp -o genion_1GHZ_water_ion.tpr -maxwarn 3
-Add ions to the system with an ionic concentration of 0.15 M , sytem charge = 8.0 water num= 8060
Add ions : NA : 22   CL : 30
gmx genion -s genion_1GHZ_water_ion.tpr -p 1GHZ_water_ion.top -o 1GHZ_water_ion.gro -np 22 -pname NA -nn 30 -nname CL
name         : 1GHZ
sim_name     : genion_1GHZ_water_ion
coor_file    : data_lig_trypsin/sys_top/1GHZ_water_ion.gro
top_file     : data_lig_trypsin/sys_top/1GHZ_water_ion.top
tpr          : data_lig_trypsin/sys_top/genion_1GHZ_water_ion.tpr
mdp          : ../../gromacs_py/template/mini.mdp
xtc          : data_lig_trypsin/prot_em/1GHZ.trr
edr          : data_lig_trypsin/prot_em/1GHZ.edr
log          : data_lig_trypsin/prot_em/1GHZ.log
nt  

In [12]:
md_sys.em_2_steps(out_folder=os.path.join(DATA_OUT, 'sys_em'),
                  no_constr_nsteps=10000,
                  constr_nsteps=10000,
                  posres="",maxwarn=3)

-Create the tpr file Init_em_1GHZ.tpr
gmx grompp -f Init_em_1GHZ.mdp -c ../sys_top/1GHZ_water_ion.gro -r ../sys_top/1GHZ_water_ion.gro -p ../sys_top/1GHZ_water_ion.top -po out_Init_em_1GHZ.mdp -o Init_em_1GHZ.tpr -maxwarn 3
-Launch the simulation Init_em_1GHZ.tpr
gmx mdrun -s Init_em_1GHZ.tpr -deffnm Init_em_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file 1GHZ.tpr
gmx grompp -f 1GHZ.mdp -c Init_em_1GHZ.gro -r Init_em_1GHZ.gro -p ../sys_top/1GHZ_water_ion.top -po out_1GHZ.mdp -o 1GHZ.tpr -maxwarn 3
-Launch the simulation 1GHZ.tpr
gmx mdrun -s 1GHZ.tpr -deffnm 1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [13]:
mdp_options = {'nsteps': 20000,
               'define': '-DPOSRES', 'dt': 0.0005,
               'tc-grps': 'Protein 120 Water_and_ions',
               'tau_t': '0.1 0.1 0.1',
               'ref_t' : '310 310 310'}

equi_template_mdp = os.path.join(gmx.GROMACS_MOD_DIRNAME,
                                 "template/equi.mdp")
md_sys.run_md_sim(out_folder=os.path.join(DATA_OUT, "sys_short_equi"),
                  name="equi_HA_short_" + PDB_ID,
                  mdp_template=equi_template_mdp,
                  mdp_options=mdp_options, maxwarn=3)

-Create the tpr file equi_HA_short_1GHZ.tpr
gmx grompp -f equi_HA_short_1GHZ.mdp -c ../sys_em/1GHZ.gro -r ../sys_em/1GHZ.gro -p ../sys_top/1GHZ_water_ion.top -po out_equi_HA_short_1GHZ.mdp -o equi_HA_short_1GHZ.tpr -maxwarn 3
-Launch the simulation equi_HA_short_1GHZ.tpr
gmx mdrun -s equi_HA_short_1GHZ.tpr -deffnm equi_HA_short_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [15]:
HA_time = 0.25
CA_time = 0.5
CA_LOW_time = 1.0
dt = 0.002
dt_HA = 0.001

md_sys.equi_three_step(out_folder=os.path.join(DATA_OUT, 'sys_equi'),
                                     nsteps_HA=1000 * HA_time / dt_HA,
                                     nsteps_CA=1000 * CA_time / dt,
                                     nsteps_CA_LOW=1000 * CA_LOW_time / dt,
                                     dt=dt, dt_HA=dt_HA, maxwarn=3)


-Create the tpr file equi_HA_1GHZ.tpr
gmx grompp -f equi_HA_1GHZ.mdp -c ../../sys_short_equi/equi_HA_short_1GHZ.gro -r ../../sys_short_equi/equi_HA_short_1GHZ.gro -p ../../sys_top/1GHZ_water_ion.top -po out_equi_HA_1GHZ.mdp -o equi_HA_1GHZ.tpr -maxwarn 3
-Launch the simulation equi_HA_1GHZ.tpr
gmx mdrun -s equi_HA_1GHZ.tpr -deffnm equi_HA_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file equi_CA_1GHZ.tpr
gmx grompp -f equi_CA_1GHZ.mdp -c ../00_equi_HA/equi_HA_1GHZ.gro -r ../../sys_short_equi/equi_HA_short_1GHZ.gro -p ../../sys_top/1GHZ_water_ion.top -po out_equi_CA_1GHZ.mdp -o equi_CA_1GHZ.tpr -maxwarn 3
-Launch the simulation equi_CA_1GHZ.tpr
gmx mdrun -s equi_CA_1GHZ.tpr -deffnm equi_CA_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright



-Create the tpr file equi_CA_LOW_1GHZ.tpr
gmx grompp -f equi_CA_LOW_1GHZ.mdp -c ../01_equi_CA/equi_CA_1GHZ.gro -r ../../sys_short_equi/equi_HA_short_1GHZ.gro -p ../../sys_top/1GHZ_water_ion.top -po out_equi_CA_LOW_1GHZ.mdp -o equi_CA_LOW_1GHZ.tpr -maxwarn 3
-Launch the simulation equi_CA_LOW_1GHZ.tpr
gmx mdrun -s equi_CA_LOW_1GHZ.tpr -deffnm equi_CA_LOW_1GHZ -nt 0 -ntmpi 0 -nsteps -2 -nocopyright


In [16]:
view = md_sys.view_traj()
view

NGLWidget(max_frame=100)